In [ ]:
import re, pymorphy2

with open('neg_rezult.txt') as f:
    dic_list = f.readlines()
dic_list = [x.strip('\n') for x in dic_list]
dic = {}
for x in dic_list:
    str_list = x.split()
    if len(str_list) == 3:
        #print(str_list[0], str_list[2])        
        dic[str_list[0]] = [str_list[1], float(str_list[2])]
#print(dic)

morph = pymorphy2.MorphAnalyzer()
def adj(word):
    p = morph.parse(word)[0]
    return p.tag.POS

#Read the file
with open('/home/alex/Загрузки/rus_news_2010_1M-sentences.txt') as f:
    text = f.read()

#Delete digits    
text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text)

#Split text into sentences and remove empty elements from list
sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
sentences = list(filter(None, sentences))
#print(sentences)

sent_list = []
for x in sentences:  
    #Delete commas, points and other and make lowercase
    delete = re.compile(u'\W+?', re.UNICODE)
    x = delete.sub(' ', x)
    #Delete empty elements   
    x = x.lower()
    #print(x)
    
    #Split into words
    str_list = x.split()
    
    #List from sentenses (words in list)
    sent_list.append(str_list)

print(sent_list)    
    
dic_new = {}
for word in dic.keys():
    if dic[word][0] == 'A':
        for x in sent_list:            
            if word in x:                
                pos = x.index(word)               
                pos_m = pos - 1
                pos_p = pos + 1
                #Into the left with "and"
                if (pos_m > 0) and (x[pos_m] == 'и'):
                    if (pos_m - 1 >= 0) and (adj(x[pos_m - 1]) == "ADJF"):
                        if x[pos_m - 1] not in dic.keys():
                            if x[pos_m - 1] not in dic_new.keys():
                                dic_new[x[pos_m - 1]] = ['A', dic[word][1]]
                                #print(word, dic_new[x[pos_m - 1]])
                #Into the left with "but"            
                if (pos_m > 0) and (x[pos_m] == 'но'):                   
                    if (pos_m - 1 >= 0) and (adj(x[pos_m - 1]) == "ADJF"):
                        if x[pos_m - 1] not in dic.keys():
                            if x[pos_m - 1] not in dic_new.keys():
                                dic_new[x[pos_m - 1]] = ['A', -dic[word][1]]
                                #print(word, dic_new[x[pos_m - 1]])
                #Into the right with "and"
                if (pos_p < len(x)) and (x[pos_p] == 'и'):
                    if (pos_p + 1 < len(x)) and (adj(x[pos_p + 1]) == "ADJF"):
                        if x[pos_p + 1] not in dic.keys():
                            if x[pos_p + 1] not in dic_new.keys():
                                dic_new[x[pos_p + 1]] = ['A', dic[word][1]]
                                #print(word, dic_new[x[pos_p + 1]])
                #Into the right with "but"
                if (pos_p < len(x)) and (x[pos_p] == 'но'):
                    if (pos_p + 1 < len(x)) and (adj(x[pos_p + 1]) == "ADJF"):
                        if x[pos_p + 1] not in dic.keys():
                            if x[pos_p + 1] not in dic_new.keys():
                                dic_new[x[pos_p + 1]] = ['A', -dic[word][1]]
                                #print(word, dic_new[x[pos_p + 1]])
                            
print(dic_new)
with open ('dic_ext_ru.txt', 'w') as fp:
    for word in dic_new.keys():
        fp.write("%s %s %s\n" % (word, dic_new[word][0], dic_new[word][1]))                
                
        